In [9]:
# ============================================================
#  동치 관계 판별 및 폐포 생성 프로그램
#
#  - 집합 A = {1,2,3,4,5} 에 대한 관계행렬(5×5)을 입력 받아
#    반사/ 대칭/ 추이 여부 판별
#  - 동치관계이면 동치류(equivalence classes) 출력
#  - 동치관계가 아닐 경우 반사/대칭/추이 폐포를 각각 생성하여
#    변환 전/후 행렬을 모두 출력
#  - 모든 폐포를 적용해 동치관계로 만든 후 다시 판별하여 동치류 출력
# ============================================================

from collections import deque

# 행렬 크기 (집합 A={1,2,3,4,5})
N = 5

# 출력 시 사용하기 위한 요소 목록
ELEMENTS = [1,2,3,4,5]


# ------------------------------------------------------------
# (1) 관계행렬 입력 함수
# ------------------------------------------------------------
def input_relation():
    print("관계행렬을 5행(각 행은 5개의 0/1 숫자, 공백으로 구분)으로 입력하세요.")
    print("예: 1 0 0 1 0")

    mat = []
    for r in range(N):
        while True:
            # 사용자에게 r+1번째 행 입력 요청
            line = input(f"행 {r+1}: ").strip()
            parts = line.split()

            # 입력 개수 오류 처리
            if len(parts) != N:
                print("오류: 반드시 5개의 값을 입력해야 합니다.")
                continue

            # 입력을 정수 0 또는 1로 변환
            try:
                row = [1 if int(x) != 0 else 0 for x in parts]
            except:
                print("오류: 0 또는 1만 입력해야 합니다.")
                continue

            # 0/1 이외의 값 방지
            if any(x not in (0,1) for x in row):
                print("오류: 0 또는 1만 입력해야 합니다.")
                continue

            mat.append(row)
            break

    return mat


# ------------------------------------------------------------
# 행렬 출력 함수 (타이틀 포함)
# ------------------------------------------------------------
def print_matrix(mat, title=None):
    if title:
        print("==", title, "==")
    for r in mat:
        print(" ".join(str(x) for x in r))
    print()


# ------------------------------------------------------------
# (2) 관계의 성질 판별 함수들
# ------------------------------------------------------------

# 반사적(reflexive) 판별: 모든 i에 대해 (i,i)=1 인가?
def is_reflexive(mat):
    for i in range(N):
        if mat[i][i] != 1:
            return False
    return True

# 대칭적(symmetric) 판별: (i,j)==(j,i) ?
def is_symmetric(mat):
    for i in range(N):
        for j in range(N):
            if mat[i][j] != mat[j][i]:
                return False
    return True

# 추이적(transitive) 판별: (i,j)=1 and (j,k)=1 이면 (i,k)=1 ?
def is_transitive(mat):
    for i in range(N):
        for j in range(N):
            if mat[i][j]:  # i→j가 있으면
                for k in range(N):
                    if mat[j][k] and not mat[i][k]:
                        # i→j, j→k인데 i→k는 없음 → 추이 위배
                        return False
    return True


# ------------------------------------------------------------
# (3) 반사 / 대칭 / 추이 폐포 생성 함수들
# ------------------------------------------------------------

# 반사 폐포: 모든 (i,i)를 1로 만든다
def reflexive_closure(mat):
    new = [row[:] for row in mat]
    for i in range(N):
        new[i][i] = 1
    return new

# 대칭 폐포: (i,j)=1 또는 (j,i)=1 이면 둘 다 1로 만든다
def symmetric_closure(mat):
    new = [row[:] for row in mat]
    for i in range(N):
        for j in range(N):
            if mat[i][j] or mat[j][i]:
                new[i][j] = 1
                new[j][i] = 1
    return new

# 추이 폐포
def transitive_closure(mat):
    new = [row[:] for row in mat]
    for k in range(N):
        for i in range(N):
            if new[i][k]:
                for j in range(N):
                    # i→k & k→j → i→j
                    if new[k][j]:
                        new[i][j] = 1
    return new


# ------------------------------------------------------------
# (4) 동치류 계산: 연결된 모든 노드 묶기
# ------------------------------------------------------------
def equivalence_classes(mat):
    visited = [False] * N
    classes = []

    for i in range(N):
        if not visited[i]:
            component = []
            queue = deque([i])
            visited[i] = True

            while queue:
                u = queue.popleft()
                component.append(u+1)  # 요소 번호(1~5)

                for v in range(N):
                    # mat[u][v] 또는 mat[v][u] 둘 중 하나라도 1이면 같은 집합으로 묶음
                    if (mat[u][v] or mat[v][u]) and not visited[v]:
                        visited[v] = True
                        queue.append(v)

            classes.append(sorted(component))

    # 동치류는 작은 숫자가 먼저 오도록 정렬
    classes.sort(key=lambda c: c[0])
    return classes


# ------------------------------------------------------------
# 관계 판별 및 동치류 출력 + 메시지 출력
# ------------------------------------------------------------
def check_and_report(mat, label="입력 관계"):
    print_matrix(mat, title=label)

    r = is_reflexive(mat)
    s = is_symmetric(mat)
    t = is_transitive(mat)

    print(f"판별 결과: 반사 = {r}, 대칭 = {s}, 추이 = {t}")

    # 모든 성질이 참이면 동치관계
    if r and s and t:
        print("→ 이 관계는 **동치관계** 입니다.")
        classes = equivalence_classes(mat)
        print("동치류:")
        for cl in classes:
            print("{" + ", ".join(str(x) for x in cl) + "}")
    else:
        print("→ 이 관계는 동치관계가 아닙니다.")

    print("-" * 40)
    return r, s, t


# ------------------------------------------------------------
# (5) 폐포 적용 및 전/후 행렬 출력 + 최종 동치관계 판별
# ------------------------------------------------------------
def apply_closures_and_report(initial_mat):

    # 먼저 현재 관계의 성질 판별
    r, s, t = is_reflexive(initial_mat), is_symmetric(initial_mat), is_transitive(initial_mat)
    current = [row[:] for row in initial_mat]

    # ---------------- 반사 폐포 ----------------
    if not r:
        rc = reflexive_closure(current)
        print_matrix(current, "반사 폐포 적용 전")
        print_matrix(rc, "반사 폐포 적용 후")
    else:
        print("반사 조건 이미 만족(폐포 불필요)")

    # ---------------- 대칭 폐포 ----------------
    if not s:
        sc = symmetric_closure(current)
        print_matrix(current, "대칭 폐포 적용 전")
        print_matrix(sc, "대칭 폐포 적용 후")
    else:
        print("대칭 조건 이미 만족(폐포 불필요)")

    # ---------------- 추이 폐포 ----------------
    if not t:
        tc = transitive_closure(current)
        print_matrix(current, "추이 폐포 적용 전")
        print_matrix(tc, "추이 폐포 적용 후")
    else:
        print("추이 조건 이미 만족(폐포 불필요)")

    # --------------------------------------------------------
    # (5-1) 전체 폐포 적용 (반사 → 대칭 → 추이)
    # --------------------------------------------------------
    eq_mat = transitive_closure(
                symmetric_closure(
                    reflexive_closure(initial_mat)
                )
            )

    print_matrix(eq_mat, "반사·대칭·추이 폐포 적용 결과(동치관계)")

    # 최종적으로 동치관계 판별 및 동치류 출력
    check_and_report(eq_mat, "폐포 적용 후 관계")


# ------------------------------------------------------------
# (7) 예제 자동 실행
# ------------------------------------------------------------
def demo_examples():
    print("=== 예제 1: 이미 동치관계인 관계 ===")
    mat1 = [
        [1,1,0,0,0],
        [1,1,0,0,0],
        [0,0,1,0,0],
        [0,0,0,1,1],
        [0,0,0,1,1]
    ]
    check_and_report(mat1, "예제 1")

    print("\n=== 예제 2: 동치관계가 아닌 관계 (폐포 확인) ===")
    mat2 = [
        [0,1,0,0,0],
        [0,0,1,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0],
        [0,0,0,0,0]
    ]
    check_and_report(mat2, "예제 2 (원본)")
    apply_closures_and_report(mat2)


# ------------------------------------------------------------
# (8) 메인 함수
# ------------------------------------------------------------
def main():
    print("==== 동치 관계 판별 및 폐포 생성 프로그램 ====")

    choice = ""
    while choice not in ("1","2","3"):
        print("1) 직접 관계행렬 입력")
        print("2) 예제 실행")
        print("3) 종료")
        choice = input("선택: ")

    if choice == "1":
        R = input_relation()
        check_and_report(R, "입력 관계")
        apply_closures_and_report(R)

    elif choice == "2":
        demo_examples()

    else:
        print("프로그램 종료.")

# 프로그램 실행 시작
if __name__ == "__main__":
    main()


==== 동치 관계 판별 및 폐포 생성 프로그램 ====
1) 직접 관계행렬 입력
2) 예제 실행
3) 종료
선택: 2
=== 예제 1: 이미 동치관계인 관계 ===
== 예제 1 ==
1 1 0 0 0
1 1 0 0 0
0 0 1 0 0
0 0 0 1 1
0 0 0 1 1

판별 결과: 반사 = True, 대칭 = True, 추이 = True
→ 이 관계는 **동치관계** 입니다.
동치류:
{1, 2}
{3}
{4, 5}
----------------------------------------

=== 예제 2: 동치관계가 아닌 관계 (폐포 확인) ===
== 예제 2 (원본) ==
0 1 0 0 0
0 0 1 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0

판별 결과: 반사 = False, 대칭 = False, 추이 = False
→ 이 관계는 동치관계가 아닙니다.
----------------------------------------
== 반사 폐포 적용 전 ==
0 1 0 0 0
0 0 1 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0

== 반사 폐포 적용 후 ==
1 1 0 0 0
0 1 1 0 0
0 0 1 0 0
0 0 0 1 0
0 0 0 0 1

== 대칭 폐포 적용 전 ==
0 1 0 0 0
0 0 1 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0

== 대칭 폐포 적용 후 ==
0 1 0 0 0
1 0 1 0 0
0 1 0 0 0
0 0 0 0 0
0 0 0 0 0

== 추이 폐포 적용 전 ==
0 1 0 0 0
0 0 1 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0

== 추이 폐포 적용 후 ==
0 1 1 0 0
0 0 1 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0

== 반사·대칭·추이 폐포 적용 결과(동치관계) ==
1 1 1 0 0
1 1 1 0 0
1 1 1 0 0
0 0 0 1 0
0 0 0 0 1

== 폐포 적용 후 관계 ==
1 1 1 0 0